In [ ]:
import pandas as pd
import numpy as np

**파일 불러오기**

In [ ]:
train = pd.read_csv("../input/instacart-market-basket-analysis/order_products__train.csv")
prior = pd.read_csv("../input/instacart-market-basket-analysis/order_products__prior.csv")
orders = pd.read_csv("../input/instacart-market-basket-analysis/orders.csv")
products = pd.read_csv("../input/instacart-market-basket-analysis/products.csv")
aisles = pd.read_csv("../input/instacart-market-basket-analysis/aisles.csv")
departments = pd.read_csv("../input/instacart-market-basket-analysis/departments.csv")

train : 0~1384616 행으로 구성, order_id 는 1~3421070까지 랜덤하게 있음. order_id / product_id / add_to_cart_order / reordered 로 구성.

prior : 0~32434488 행으로 구성, order_id 는 2~342183까지 랜덤하게 있음. order_id / product_id / add_to_cart_order / reordered 로 구성.

orders : 0~3421082 행으로 구성, user_id는 1~206209까지 오름차순으로 정렬. order_id / user_id / eval_set / order_number / order_dow / order_hour_of_day / days_since_prior_order 로 구성.

products : 0~49687 행으로 구성, product_id 는 1~49688까지 오름차순으로 정렬. product_id / product_name / aisle_id / department_id 로 구성.

aisles : 0~133 행으로 구성, aisle_id 는 1~134까지 오름차순으로 정렬. aisle_id / aisle 로 구성.

departments : 0~20 행으로 구성, 1~21까지 오름차순으로 정렬, department_id / department 로 구성.






nan값 확인하는 방법

In [ ]:
for col in orders.columns: 
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (orders[col].isnull().sum() / orders[col].shape[0]))
    print(msg)

최빈값 구하기

In [ ]:
orders['order_dow'].value_counts().idxmax()

In [ ]:
orders['order_hour_of_day'].value_counts().idxmax()

orders : 
총 구매기록을 통계적으로 살펴보았을 때, 0요일과 10시에 가장 많이 구매를 함.
2번째로 많은 구매요일은 1요일, 11시이다.
4요일에 3시에 가장 적게 구매함.

In [ ]:
orders.info('order_id')

In [ ]:
order_id = prior.loc[:,["order_id"]]
print(order_id)

In [ ]:
order_id_1 = train.loc[:,["order_id"]]
print(order_id_1)

In [ ]:
order_id.duplicated()

In [ ]:
order_id = order_id.drop_duplicates()

In [ ]:
print(order_id)

In [ ]:
print(len(order_id))

In [ ]:
order_id_1.duplicated()

In [ ]:
order_id_1 = order_id_1.drop_duplicates()

In [ ]:
print(order_id_1)

In [ ]:
print(len(order_id_1))

In [ ]:
order_len = len(order_id) + len(order_id_1)
print(order_len)

In [ ]:
A = pd.concat([order_id, order_id_1])

In [ ]:
A.duplicated()

In [ ]:
A = A.drop_duplicates()

In [ ]:
print(len(A))

train 과 prior이 겹치지 않으므로 train으로 데이터를 합친다.

In [ ]:
prior = pd.concat([train, prior])

In [ ]:
prior.head()

In [ ]:
print(len(prior))

In [ ]:
print(products[49301:])

****prior1 ****=> prior과 train을 결합하고, products를 결합함.
order_id / product_id / add_to_cart_order / reordered / product_name / aisle_id / department_id 로 구성되어 있음.**

In [ ]:
prior1 = prior.join(products.set_index('product_id'), on='product_id')

depart_aisle => department_id에 해당하는 product_id를 알 수 있음.

In [ ]:
depart_aisle = pd.merge(departments, products, how='left')

In [ ]:
print(depart_aisle)

In [ ]:
depart_aisle = depart_aisle[['department_id', 'aisle_id']]

In [ ]:
depart_aisle.duplicated()

In [ ]:
depart_aisle = depart_aisle.drop_duplicates()

In [ ]:
print(depart_aisle)

In [ ]:
depart_aisle = depart_aisle.sort_values(by=['department_id','aisle_id'])

In [ ]:
print(depart_aisle)

product_name에서 가장 많이 팔린 3가지

In [ ]:
from collections import Counter as cc

pro_name = cc(prior1['product_name'])
print(pro_name.most_common()[:10])

prior1과 orders의 order_id를 기준으로 결합하기

In [ ]:
#All_orders = pd.merge(prior1, orders, how='left')

In [ ]:
#All_orders = All_orders.sort_values(by=['order_id','order_number','add_to_cart_order'])

In [ ]:
#All_orders.head(10)

In [ ]:
#All_orders.to_csv("All_orders.csv",mode='w')

In [ ]:
prior2 = prior1[['product_name','aisle_id']]

In [ ]:
prior2.head()

In [ ]:
#prior2 = prior2.sort_values(by=['product_name'])

In [ ]:
#prior2.head()

In [ ]:
print(prior2[prior2['product_name'].isin([])].count())

In [ ]:
product_1 = prior2.groupby(['product_name', 'aisle_id'])['product_name'].count().reset_index(name="product_count")

In [ ]:
product_1 = pd.DataFrame(product_1)

In [ ]:
product_1.head()

In [ ]:
product_1 = product_1.sort_values(by=['aisle_id','product_count'],ascending=[True,False])

In [ ]:
product_1.head()

In [ ]:
aisle_sum = product_1['product_count'].groupby(product_1['aisle_id']).sum().reset_index(name="aisle_sum")

In [ ]:
aisle_sum = pd.DataFrame(aisle_sum)

소분류에서는 24번이 가장 높게 나옴.

In [ ]:
aisle_sum = aisle_sum.sort_values(by=['aisle_sum'], ascending=[False])

In [ ]:
aisle_sum.head()

In [ ]:
prior1.head()

In [ ]:
prior1.columns[::]

In [ ]:
depart_count = prior1[['product_name','department_id']]

In [ ]:
depart_count = depart_count['product_name'].groupby(depart_count['department_id']).count().reset_index(name="depart_count")

In [ ]:
depart_count = depart_count.sort_values(by=['depart_count'],ascending=[False])

In [ ]:
depart_count.head(10)